In [2]:
import datetime
import pandas as pd
import numpy as np
import geopy as gp
import xgboost as xgb
import nltk
import random
import time
import zipfile


from geopy.distance import vincenty
from sklearn.cross_validation import train_test_split
from operator import itemgetter
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords

In [3]:

def get_features(train, test):
    trainval = list(train.columns.values)
    testval = list(test.columns.values)
    output = intersect(trainval, testval)
    output.remove('itemID_1')
    output.remove('itemID_2')
    return output


def intersect(a, b):
    return list(set(a) & set(b))

In [4]:


def prep_dataset(is_train, dataset_pairs_path, dataset_info_path, tfidfs_description_path, tfidfs_title_path):
    start_time = time.time()

    types1 = {
        'itemID_1': np.dtype(int),
        'itemID_2': np.dtype(int),
        'isDuplicate': np.dtype(int),
        'generationMethod': np.dtype(int),
    }

    types2 = {
        'itemID': np.dtype(int),
        'categoryID': np.dtype(int),
        'title': np.dtype(str),
        'description': np.dtype(str),
        'images_array': np.dtype(str),
        'attrsJSON': np.dtype(str),
        'price': np.dtype(float),
        'locationID': np.dtype(int),
        'metroID': np.dtype(float),
        'lat': np.dtype(float),
        'lon': np.dtype(float),
    }

    pairs = pd.read_csv(dataset_pairs_path, dtype=types1)

    # Add 'id' column for easy merge
    items = pd.read_csv(dataset_info_path, dtype=types2)
    items.fillna(-1, inplace=True)
    location = pd.read_csv("input/Location.csv")
    category = pd.read_csv("input/Category.csv")

    train = pairs
    
    if is_train:
        train = train.drop(['generationMethod'], axis=1)

    print('Add text features...')
    # Dodaj różnice i sumy
    #train['len_title'] = items['title'].str.len()
    #train['len_description'] = items['description'].str.len()

    print('Merge item 1...')
    item1 = items[['itemID', 'categoryID', 'description', 'price', 'locationID', 'metroID', 'lat', 'lon']]
    item1 = pd.merge(item1, category, how='left', on='categoryID', left_index=True)
    item1 = pd.merge(item1, location, how='left', on='locationID', left_index=True)

    item1 = item1.rename(
        columns={
            'itemID': 'itemID_1',
            'categoryID': 'categoryID_1',
            'description': 'description_1',
            'parentCategoryID': 'parentCategoryID_1',
            'price': 'price_1',
            'locationID': 'locationID_1',
            'regionID': 'regionID_1',
            'metroID': 'metroID_1',
            'lat': 'lat_1',
            'lon': 'lon_1'
        }
    )

    # Add item 1 data
    train = pd.merge(train, item1, how='left', on='itemID_1', left_index=True)

    print('Merge item 2...')
    item2 = items[['itemID', 'categoryID', 'description', 'price', 'locationID', 'metroID', 'lat', 'lon']]
    item2 = pd.merge(item2, category, how='left', on='categoryID', left_index=True)
    item2 = pd.merge(item2, location, how='left', on='locationID', left_index=True)

    item2 = item2.rename(
        columns={
            'itemID': 'itemID_2',
            'categoryID': 'categoryID_2',
            'description': 'description_2',
            'parentCategoryID': 'parentCategoryID_2',
            'price': 'price_2',
            'locationID': 'locationID_2',
            'regionID': 'regionID_2',
            'metroID': 'metroID_2',
            'lat': 'lat_2',
            'lon': 'lon_2'
        }
    )

    # Add item 2 data
    train = pd.merge(train, item2, how='left', on='itemID_2', left_index=True)

    # Create same arrays
    print('Create same arrays')

    
    # Create distance
    def geopydistance(a, b, c, d):
        p1 = gp.Point([a, b])
        p2 = gp.Point([c, d])
        return vincenty(p1,p2).kilometers
    vecfunc = np.vectorize(geopydistance)
    
    train['distance'] = vecfunc(train['lat_1'], train['lon_1'], train['lat_2'], train['lon_2'])

    train['price_abs'] = np.true_divide(np.abs(np.subtract(train['price_1'], train['price_2'])),
                                        np.add(train['price_1'], train['price_2']))
    train['price_sum'] = np.add(train['price_1'], train['price_2'])
    train['price_diff'] = np.abs(np.subtract(train['price_1'], train['price_2']))
    train['lat_diff'] = np.abs(np.subtract(train['lat_1'], train['lat_2']))
    train['lon_diff'] = np.abs(np.subtract(train['lon_1'], train['lon_2']))
    train['lat_sum'] = np.add(train['lat_1'], train['lat_2'])       
    train['lon_sum'] = np.add(train['lon_1'], train['lon_2'])   
                               
    # Create categories!
    def same(a,b):
        if a == b:
            return a
        else:
            if isinstance(a, str):
                return "different"
            else:
                return -1.0
    vecfunc = np.vectorize(same)
                               
    train['category'] = vecfunc(train['categoryID_1'], train['categoryID_2'])
    train['metro'] = vecfunc(train['metroID_1'], train['metroID_2'])
    train['region'] = vecfunc(train['regionID_1'], train['regionID_2'])
    train['location'] = vecfunc(train['locationID_1'], train['locationID_2'])
    train['lat_same'] = vecfunc(train['lat_1'], train['lat_2'])
    train['lon_same'] = vecfunc(train['lon_1'], train['lon_2'])
          
    # Categories

    # Add tfidfs
    train['title_dist'] = np.loadtxt(tfidfs_title_path, usecols=(1,), delimiter=",")
    train['description_dist'] = np.loadtxt(tfidfs_description_path, usecols=(1,), delimiter=",")
    
    print('Create train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    
    # ITEM IDS ARE REMOVED LATER
    train.drop(['description_1', 
                'description_2',
                'price_1',
                'price_2',
                'lat_1',
                'lat_2',
                'lon_1',
                'lon_2',
                'metroID_1',
                'metroID_2',
                'regionID_1',
                'regionID_2',
                'locationID_2',
                'locationID_1',
                'parentCategoryID_2',
                'parentCategoryID_1',
                'categoryID_2',
                'categoryID_1',
                ], axis=1, inplace=True)
    return train

def read_test_train(from_disk=False):
    
    train = prep_dataset(True, "input/ItemPairs_train.csv", "input/ItemInfo_train.csv",
                               "similarities/train_description_stopwords_cosine.csv",
                               "similarities/train_title_stopwords_cosine.csv")
    train.fillna(-1, inplace=True)
    train.to_csv('input/train_merged.csv')


    test = prep_dataset(False, "input/ItemPairs_test.csv", "input/ItemInfo_test.csv",
                               "similarities/test_description_stopwords_cosine.csv",
                               "similarities/test_title_stopwords_cosine.csv")
    test.fillna(-1, inplace=True)
    test.to_csv('input/test_merged.csv')

    features = get_features(train, test)
    return train, test, features


In [5]:

train, test, features = read_test_train()
print('Length of train: ', len(train))
print('Length of test: ', len(test))
print('Features [{}]: {}'.format(len(features), sorted(features)))


Add text features...
Merge item 1...
Merge item 2...
Create same arrays
Create train data time: 113.81 seconds
Add text features...
Merge item 1...
Merge item 2...
Create same arrays
Create train data time: 47.98 seconds
('Length of train: ', 2991396)
('Length of test: ', 1044196)
Features [16]: ['category', 'description_dist', 'distance', 'lat_diff', 'lat_same', 'lat_sum', 'location', 'lon_diff', 'lon_same', 'lon_sum', 'metro', 'price_abs', 'price_diff', 'price_sum', 'region', 'title_dist']
